# Modifications ijN1463 model

In [1]:
#Script for PET consuming microorganism (previously TPHA consuming)
#Script to modify the model iJN1463.xml for FLYCOP simulation
#07/12/20
#Ana del Ramo Galián
from cobra import Reaction
from cobra import Metabolite
import cobra as cb
import cometspy as c
model=cb.io.read_sbml_model("../../models/modelManoli.xml")
model

Using license file /home/chanle/software/gurobi900/linux64/gurobi.lic
Academic license - for non-commercial use only


Name,model
Memory address,0x07fdc01fe8828
Number of metabolites,2008
Number of reactions,2625
Number of groups,0
Objective expression,1.0*BiomassKT2440_ME - 1.0*BiomassKT2440_ME_reverse_0e45e
Compartments,"p, c, e"


## Deleting the reaction PCADYOX

In [2]:
model.reactions.get_by_id('PCADYOX').bounds=(0,0)

## Adding PET as a carbon source

In [3]:
model.reactions.EX_glycol_e.bounds=(0,1000)
model.reactions.EX_o2_e.bounds=(-18.5,0)
model.reactions.EX_tpha_e.bounds=(0,0)
model.reactions.get_by_id('GLYCOLDHpp').bounds=(0,1000) # get by id GLYCOLDHpp, Bounds

# Deleting the glucose as a carbon source

In [4]:
model.reactions.EX_glc_e.bounds=(0,0)
model.reactions.EX_pet_e.bounds=(-5,0) # -5 pet en notebook quiero 10 en el layout de COMETS

### Creating PETase reaction and transport

# PET Hydrolase

In [74]:
#Quitar

#PETHY_reaction=Reaction('PETHY')
#PETHY_reaction.name='PET Hydrolase'
#PETHY_reaction.lower_bound=0
#PETHY_reaction.upper_bound=1000
#PETHY_reaction.add_metabolites({pet_e:-1,model.metabolites.get_by_id('h2o_e'):-1, model.metabolites.get_by_id('h_e'):1, model.metabolites.get_by_id('glycol_e'):1, model.metabolites.get_by_id('tpha_e'):1})
#PETHY_reaction.reaction

#PET Exchange (transporter)

In [75]:
#Quitar
#EX_pet_e_reaction=Reaction('EX_pet_e')
#EX_pet_e_reaction.name='PET Exchange'
#EX_pet_e_reaction.lower_bound=-5
#EX_pet_e_reaction.upper_bound=1000
#EX_pet_e_reaction.add_metabolites({pet_e:-1})
#EX_pet_e_reaction.reaction

# Creating C80aPHAtpp reaction

In [5]:
C80aPHAtpp_reaction=Reaction('C80aPHAtpp')
C80aPHAtpp_reaction.name='C80aPHA transporter periplasm'
C80aPHAtpp_reaction.lower_bound=0
C80aPHAtpp_reaction.upper_bound=1000
c80apha_p = Metabolite('C80aPHA_p', formula='C8H15O3R', name='C80_Medium_chain_length_aliphatic_Polyhydroxyalkanoate_p',compartment='p')
c80apha_e = Metabolite('C80aPHA_e', formula='C8H15O3R', name='C80_Medium_chain_length_aliphatic_Polyhydroxyalkanoate_e',compartment='e')
C80aPHAtpp_reaction.add_metabolites({model.metabolites.get_by_id('C80aPHA_c'):-1, c80apha_p:1})
C80aPHAtpp_reaction.reaction

'C80aPHA_c --> C80aPHA_p'

# Creating C80aPHAtex reaction

In [6]:
C80aPHAtex_reaction=Reaction('C80aPHAtex')
C80aPHAtex_reaction.name='C80aPHA transporter extracellular'
C80aPHAtex_reaction.lower_bound=0
C80aPHAtex_reaction.upper_bound=1000
C80aPHAtex_reaction.add_metabolites({c80apha_p:-1, c80apha_e:1})
C80aPHAtex_reaction.reaction

'C80aPHA_p --> C80aPHA_e'

# Creating EX_C80aPHA_e reaction

In [7]:
EX_C80aPHA_e_reaction=Reaction('EX_C80aPHA_e')
EX_C80aPHA_e_reaction.name='C80aPHA exchange'
EX_C80aPHA_e_reaction.lower_bound=0
EX_C80aPHA_e_reaction.upper_bound=1000
EX_C80aPHA_e_reaction.add_metabolites({c80apha_e:-1})
EX_C80aPHA_e_reaction.reaction

'C80aPHA_e --> '

In [8]:
model.add_reactions([C80aPHAtex_reaction, C80aPHAtpp_reaction, EX_C80aPHA_e_reaction])

# Eliminate DM_C80aPHA_c reaction

In [9]:
model.reactions.get_by_id('DM_C80aPHA').bounds=(0,0)

In [10]:
for reaction in ['RHACOAE80','PHADPC80','CYSS','NACODA','ALDD2x','ABUTD','ALAR','ALATA_L','FORGLUIH2','ICDHyr','ICL','MALS','MCITL2','OARGDC','ORNCD','PDHbr','PDHcr','PPCSCT','RBK','SUCOAS','UPPN']:
    model.reactions.get_by_id(reaction).bounds=(0,0)

In [11]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0002983,0,0.00%
cl_e,EX_cl_e,0.0002983,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0001988,0,0.00%
cu2_e,EX_cu2_e,0.0001988,0,0.00%
fe2_e,EX_fe2_e,0.001037,0,0.00%
k_e,EX_k_e,0.01119,0,0.00%
mg2_e,EX_mg2_e,0.0004972,0,0.00%
mobd_e,EX_mobd_e,0.0001988,0,0.00%
na1_e,EX_na1_e,0.0002486,0,0.00%
nh4_e,EX_nh4_e,0.7502,0,0.00%


In [36]:

model.reactions.GLYCOLtex.bounds=(-1000,1000)
model.metabolites.get_by_id('glycol_e').summary()
model.reactions.EX_glycol_e.bounds=(0,0.2)
model.reactions.GLYCOLtex.bounds=(-0.2,1000)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0002983,0,0.00%
cl_e,EX_cl_e,0.0002983,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0001988,0,0.00%
cu2_e,EX_cu2_e,0.0001988,0,0.00%
fe2_e,EX_fe2_e,0.001037,0,0.00%
k_e,EX_k_e,0.01119,0,0.00%
mg2_e,EX_mg2_e,0.0004972,0,0.00%
mobd_e,EX_mobd_e,0.0001988,0,0.00%
na1_e,EX_na1_e,0.0002486,0,0.00%
nh4_e,EX_nh4_e,0.7502,0,0.00%


In [43]:
model.reactions.GLYCOLtex.bounds=(-1000,1)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,5.116E-05,0,0.00%
cl_e,EX_cl_e,5.116E-05,0,0.00%
cobalt2_e,EX_cobalt2_e,3.41E-05,0,0.00%
cu2_e,EX_cu2_e,3.41E-05,0,0.00%
fe2_e,EX_fe2_e,0.0001778,0,0.00%
k_e,EX_k_e,0.001919,0,0.00%
mg2_e,EX_mg2_e,8.528E-05,0,0.00%
mobd_e,EX_mobd_e,3.41E-05,0,0.00%
na1_e,EX_na1_e,4.264E-05,0,0.00%
nh4_e,EX_nh4_e,0.1287,0,0.00%


In [29]:
dictCoumarateValue=cb.flux_analysis.variability.flux_variability_analysis(model,['EX_glycol_e'],fraction_of_optimum=1-(20/100))
dictCoumarateValue['maximum']['EX_glycol_e']


1.0138947368421003

In [ ]:
model.reactions.get_by_id('GLYCOLtex').bounds=(coumarateLimit,1000)
model.reactions.get_by_id('EX_glycol_e').bounds=(0,coumarateLimit)
model.summary()

In [17]:
comets_model=c.model(model)
comets_model.id='plasticDegradationEstrategy3b-PETase-dPCA_0_1000'

In [35]:
from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(model,"EX_glycol_e", fraction_of_optimum=0.9)
model.optimize()
print(model.summary(fva=0.9))

Objective
1.0 BiomassKT2440_ME = 0.07058705512583689

Uptake
------
Metabolite      Reaction      Flux                   Range  C-Number   C-Flux
     ca2_e      EX_ca2_e 0.0002983  [0.0002685; 0.0002983]         0    0.00%
      cl_e       EX_cl_e 0.0002983  [0.0002685; 0.0002983]         0    0.00%
 cobalt2_e  EX_cobalt2_e 0.0001988   [0.000179; 0.0002003]         0    0.00%
     cu2_e      EX_cu2_e 0.0001988   [0.000179; 0.0001988]         0    0.00%
     fe2_e      EX_fe2_e  0.001037    [0.000933; 0.004079]         0    0.00%
       k_e        EX_k_e   0.01119      [0.01007; 0.01119]         0    0.00%
     mg2_e      EX_mg2_e 0.0004972  [0.0004475; 0.0004972]         0    0.00%
    mobd_e     EX_mobd_e 0.0001988   [0.000179; 0.0002019]         0    0.00%
     na1_e      EX_na1_e 0.0002486  [0.0002237; 0.0002486]         0    0.00%
     nh4_e      EX_nh4_e    0.7502         [0.6752; 1.373]         0    0.00%
     ni2_e      EX_ni2_e 0.0001988   [0.000179; 0.0001988]         0    0.

In [84]:
def ensure_sinks_are_not_exchanges(model):
    rxn_names = ['sink_PHAg']
    for rxn in rxn_names:
        model.reactions.loc[model.reactions.REACTION_NAMES == rxn, 'EXCH'] = False
        model.reactions.loc[model.reactions.REACTION_NAMES == rxn, 'EXCH_IND'] = 0


In [85]:
ensure_sinks_are_not_exchanges(comets_model)

In [86]:

comets_model.write_comets_model()

In [87]:
my_simulation = c.comets("layout_3.txt", my_params)
my_simulation.run()

NameError: name 'my_params' is not defined